In [1]:
import LadiskDAQ

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcsetup.all_backends
import copy

import LadiskDAQ

In [3]:
task_1 = LadiskDAQ.national_instruments.NITask('task_1', sample_rate=25600)
task_1.add_channel('Channel_1', device_ind=0, channel_ind=0, sensitivity=None, sensitivity_units=None, units='pC', scale=1.)
task_1.add_channel('Channel_2', device_ind=0, channel_ind=1, sensitivity=None, sensitivity_units=None, units='pC', scale=10.)
task_1

Task name: task_1
Connected devices:
	(0) - Dev1
	(1) - Dev2_output
Channels: ['Channel_1', 'Channel_2']

In [4]:
task_out = LadiskDAQ.national_instruments.NITaskOutput('task_out', sample_rate=10000)
task_out.add_channel('out1', 1, 0, min_val=-4, max_val=4)
task_out.add_channel('out2', 1, 1, min_val=-4, max_val=4)

In [5]:
signal = 0.5*np.sin(np.linspace(0, 12*2*np.pi, 10000)) + 1*np.sin(np.linspace(0, 30*2*np.pi, 10000)) + 0.2*np.cos(np.linspace(0, 24*2*np.pi, 10000))
signal = np.vstack((signal, signal)).T

gen = LadiskDAQ.national_instruments.NIGeneration(task_out, signal)
# gen = LadiskDAQ.NIGenerator('ldaq_out', signal)

In [6]:
acq = LadiskDAQ.national_instruments.NIAcquisition(task_1, acquisition_name="NI_task")
# acq1 = LadiskDAQ.NIAcquisition('ldaq_in', acquisition_name="NI_task")

In [7]:
# acq1 = LadiskDAQ.flir.FLIRThermalCamera('FLIR')
# acq1 = LadiskDAQ.BaslerCamera('FLIR', sample_rate=30, subsample=3)

In [8]:
vis = LadiskDAQ.Visualization(sequential_plot_updates=True)

vis.add_lines((0, 0), 'NI_task', [0], refresh_rate=100, nth='auto')
vis.add_lines((1, 0), 'NI_task', [0, 1], refresh_rate=100, nth='auto')
vis.add_lines((1, 1), 'NI_task', [0], refresh_rate=500, nth='auto')
vis.add_lines((1, 1), 'NI_task', [0, 1], 'fft', refresh_rate=500, nth=1)

# vis.add_lines((0, 0), 'FLIR', [(100, 100)], 'fft', refresh_rate=500, nth=100)

# vis.add_image('FLIR', refresh_rate=100, colormap='gray')

# vis.config_subplot((0, 0), xlim=(0, 5), ylim=(-10, 10), colspan=2, title='Test')
# vis.config_subplot((1, 0), xlim=(0, 1), ylim=(-10, 10), title='Test')
# vis.config_subplot((1, 1), xlim=(0, 50), title='Test', t_span=1)

In [9]:
ldaq = LadiskDAQ.Core(acquisitions=[acq], generations=gen, visualization=vis)

ldaq.set_trigger(
     source = "NI_task",
     channel = 0,
     level = 100,
     duration= 1
)

ldaq.run(autoclose=False, autostart=False)